In [1]:
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("/home/dongwonshin/Desktop/Open3D/build/lib")
from py3d import *

from VolumetricVAE import *

# Dataloader

In [2]:
nyudv2_color_codes = { 
    'floor': [162, 227, 148], 
    'wall': [183, 205, 235], 
    'cabinet': [35, 131, 188], 
    'bed': [255, 196, 131], 
    'chair': [195, 197, 37], 
    'sofa': [151, 97, 84], 
    'table': [255, 162, 161], 
    'door': [218, 45, 45], 
    'window': [203, 185, 218], 
    'bookshelf': [159, 114, 196], 
    'picture': [203, 166, 158], 
    'counter': [24, 197, 214], 
    'desk': [248, 191, 216], 
    'curtain': [222, 224, 152], 
    'refrigerator': [254, 139, 10], 
    'bathtub':[231, 130, 201], 
    'shower_curtain': [168, 223, 232], 
    'toilet': [48, 170, 51], 
    'sink':[123, 139, 155], 
    'otherfuniture':[93, 93, 173] 
}
    
def rgb2nyudv2(rgb):
    
    color_dist = []
    for class_name, code in nyudv2_color_codes.items():
        color_dist.append(np.linalg.norm(np.asarray(code)-rgb))
    
    class_id = np.argmin(color_dist)
    class_name = list(nyudv2_color_codes.keys())[class_id]
    
    return class_id, class_name

In [3]:
def pcd2voxel(pcd, filter_rad=0.1, dim_size=32):
    [xc, yc, zc] = np.mean(pcd.points,0)
    [xs, ys, zs] = [xc, yc, zc] + np.ones(3)*filter_rad*dim_size/2
    [xe, ye, ze] = [xc, yc, zc] - np.ones(3)*filter_rad*dim_size/2

    voxel_volume = np.zeros((dim_size,dim_size,dim_size))
    color_volume = np.zeros((dim_size,dim_size,dim_size,3))
    for point, color in zip(pcd.points, pcd.colors):
        xi,yi,zi = (np.around(([xs, ys, zs] - point)/filter_rad)).astype(int)
        voxel_volume[xi, yi, zi] = 1
        color_volume[xi, yi, zi]= color
    
    return voxel_volume, color_volume

def colorvol2classidvol(color_volume):
    classid_volume = np.zeros([32,32,32])
    classid_volume = classid_volume.flatten()
    color_volume = color_volume.reshape([-1,3])

    for idx, color_voxel in enumerate(color_volume):
        if(not np.array_equal(color_voxel, [0,0,0])):
            class_id, calss_name = rgb2nyudv2(color_voxel*255)
            classid_volume[idx] = class_id
    #         print(class_id,calss_name)

    classid_volume = classid_volume.reshape([32,32,32])
    
    return classid_volume

def classidvol2colorvol(classid_volume):
    color_volume = np.zeros([32,32,32,3])
    color_volume = color_volume.reshape([-1,3])
    classid_volume = classid_volume.flatten()
    
    for idx, classid_voxel in enumerate(classid_volume):
        if (classid_voxel != 0):
            color_volume[idx] = list(nyudv2_color_codes.values())[int(classid_voxel)]
    
    color_volume = color_volume.reshape([32,32,32,3])
    
    return color_volume

# Model training

In [4]:
vol_vae = VolumetricVAE(_kernel_size=3, _stride=1)
# vol_vae = torch.nn.DataParallel(vol_vae, device_ids = range(torch.cuda.device_count()))
# vol_vae.cuda()

In [ ]:
optimizer = optim.Adam(vol_vae.parameters(), lr=0.0001)

epoches = 100

for epoch in range(epoches):
    for idx in range(20):
        incomp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_incomplete.ply" % idx)
        voxel_volume, color_volume = pcd2voxel(incomp_pcd)
        classid_volume = colorvol2classidvol(color_volume)
        classid_volume = np.expand_dims(classid_volume, axis=0)
        classid_volume = np.expand_dims(classid_volume, axis=0)
        classid_volume = torch.from_numpy(classid_volume).float()

        output, kl_loss = vol_vae.forward(classid_volume)

        comp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_complete.ply" % idx)
        _, target_volume = pcd2voxel(comp_pcd)
        target_volume = colorvol2classidvol(target_volume)
        target_volume = np.expand_dims(target_volume, axis=0)
        target_volume = torch.from_numpy(target_volume).long()

        recon_loss_func = torch.nn.CrossEntropyLoss()
        recon_loss = recon_loss_func(output,target_volume)
        
        loss = recon_loss + kl_loss
        
        loss.backward()
        optimizer.step()

        l = loss.data[0]
    print(l)

/home/dongwonshin/Desktop/Volumetric VAE/VolumetricVAE.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/dongwonshin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(100.9739)
tensor(61.8201)
tensor(66.8621)
tensor(41.9705)
tensor(54.9759)
tensor(48.0042)
tensor(26.4527)
tensor(23.4781)
tensor(24.3743)
tensor(17.5959)
tensor(17.3707)
tensor(17.3638)
tensor(17.4544)
tensor(18.4377)
tensor(15.3050)
tensor(17.4909)
tensor(17.0955)
tensor(17.9422)
tensor(19.0126)
tensor(14.1079)
tensor(19.8934)
tensor(23.9355)
tensor(21.7939)
tensor(25.5377)
tensor(30.4059)
tensor(23.2594)
tensor(14.3488)
tensor(20.5997)
tensor(32.5109)
tensor(36.3397)
tensor(30.8432)
tensor(22.7650)
tensor(23.3947)
tensor(31.8191)
tensor(35.7289)
tensor(31.8483)
tensor(26.0294)
tensor(30.2243)
tensor(46.1125)
tensor(54.6057)
tensor(47.6307)
tensor(31.3286)
tensor(21.4493)
tensor(26.8741)
tensor(37.3298)
tensor(44.1379)
tensor(45.3100)
tensor(40.8178)
tensor(31.9472)
tensor(21.6887)
tensor(14.7996)
tensor(14.8941)
tensor(20.2161)
tensor(26.1049)
tensor(29.7994)
tensor(30.3492)
tensor(28.4193)
tensor(26.9891)
tensor(30.4339)
tensor(35.5498)
tensor(35.2935)
tensor(27.2211)
tensor(